# 🦸🏻 Entendiendo Semantic Kernel con Héroes 🦸🏼‍♀️

Semantic Kernel ha sido creado para permitir a los desarrolladores integrar Inteligencia Artificial de una forma súper sencilla en sus aplicaciones. Para ello, proporciona un conjunto de características que te van a permitir añadir modelos, prompts, funciones nativas y memorias sin tener conocimientos profundos de IA 🥲. Es por ello que se dice que Semantic Kernel simula el cerebro 🧠 de tu aplicación.

<div style="text-align:center">
    <img src="images/semantic-kernel.png" width="15%">
</div>

## Crear un servicio de Azure Open AI 🤖 y despliegues

Antes de empezar a jugar 🛝 con Semantic Kernel necesitar tener algún servicio de los que soporta creado. A día de hoy son: [Azure Open AI](https://azure.microsoft.com/es-es/products/ai-services/openai-service), [Open AI](https://openai.com/) o [Hugging Face](https://huggingface.co/).

En este ejemplo, voy a utilizar Azure Open AI.

Por lo que, a través de Azure CLI necesito iniciar sesión:

In [ ]:
az login

Si lo necesitas, porque tienes muchos tenants y suscripciones bajo tu cuenta, puedes seleccionar la que te interese (o tenga acceso a Azure Open AI 😊) a través del siguiente comando:

In [2]:
az account set -n "Visual Studio Enterprise Subscription"

Ahora, para poder crear lo que necesitas para este notebook, establece las siguiente variables con los valores de tu preferencia:

In [3]:
$RESOURCE_GROUP="understanding-semantic-kernel"
$LOCATION="canadaeast"
$AZURE_OPEN_AI="ai-for-heroes"

Con ellas, ya puedes crear el grupo de recursos:

In [ ]:
az group create `
--name $RESOURCE_GROUP `
--location $LOCATION

Un recurso de Azure Open AI:

In [ ]:
az cognitiveservices account create `
--name $AZURE_OPEN_AI `
--custom-domain $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--kind OpenAI `
--sku S0 `
--location $LOCATION

Y por último, necesitas un despliegue de alguno de los modelos que tengas disponible. En este ejemplo voy a utilizar gpt-4:

In [ ]:
az cognitiveservices account deployment create `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--deployment-name "gpt-4" `
--model-name "gpt-4" `
--model-version "0613"  `
--model-format OpenAI `
--sku-capacity "10" `
--sku-name "Standard"

El parámetro **sku-capacity** es el que nos permitirá especificar cuántos tokens por minuto podemos mandarle a este modelo. Para poder ver cómo está el uso de tu cuota puedes utilizar este otro comando:

In [ ]:
az cognitiveservices usage list `
-l $LOCATION

Load enviroment variables with your Azure Open AI endpoint and key

In [5]:
$env:AZURE_OPEN_AI_KEY =$(az cognitiveservices account keys list `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "key1" `
--output tsv)

$env:AZURE_OPEN_AI_ENDPOINT =$(az cognitiveservices account show `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "properties.endpoint" `
--output tsv)

#dir env:AZURE_OPEN_AI_KEY
#dir env:AZURE_OPEN_AI_ENDPOINT

## Cómo empezar con Semantic Kernel

Lo primero que necesitas para poder ejecutar Semantic Kernel en este notebook es instalar la librería **Microsoft.SemanticKernel** que a día de hoy está en su versión **1.0.0-beta8** (prometo ir actualizando 🤓)

In [6]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta8

Con ella ya puedes instanciar el cerebro de tu aplicación a través de **KernelBuilder**. Este tiene un montón de conectores a los modelos y otras cosas, pero por ahora vamos a empezar con lo básico:

In [7]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using System.IO;

//Create Kernel builder
var builder = new KernelBuilder();

Dependiendo del tipo de tarea que quieras hacer puede utilizar diferentes métodos con el prefijo **With**. Por ahora, vamos a utilizar alguno de los modelos de tipo **completion** o completado.

In [8]:
builder
.WithAzureOpenAIChatCompletionService("gpt-4", Environment.GetEnvironmentVariable("AZURE_OPEN_AI_ENDPOINT"), Environment.GetEnvironmentVariable("AZURE_OPEN_AI_KEY"));
//.WithOpenAIChatCompletionService("gpt-4",(await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Open AI key")).GetClearTextPassword());

Con la configuración hecha, lo único que queda es generar el kernel con todo lo establecido:

In [9]:
var kernel = builder.Build();

## Los plugins

Los plugins son el core de Semantic Kernel. Con ellos encapsulas las capacidades de forma que estas puedan ser reutilizables, mantenibles y planificables (más adelante lo entenderás 🙃). Hay dos tipos: aquellos que consisten en plantillas de prompts llamadas **Semantic Functions** y funciones nativas del lenguaje programación elegido llamadas **Native Functions**.

<div style="text-align:center">
    <img src="images/writer-plugin-example.png" width="40%" />
</div>

### Semantic Functions

Cuando hablas con los modelos de inteligencia artificial debes hacerlo con lo que se conoce como *prompt* este puede ser desde una simple frase a algo más elaborado que nos permite que el modelo conozca no solamente lo que queremos sino el cómo lo queremos. Si echas un vistazo a la documentación oficial define este tipo de funciones como la boca 👄 y los oidos 👂🏻 de tu cerebro 🧠.

<div style="text-align:center">
    <img src="images/semantic-function-explainer.png" width="20%" />
</div>

Como parte de este repo tienes una carpeta llamada **SemanticFunctions** que tiene diferentes funciones de este tipo:

- **FunPlugin**: Nos permite pedirle al modelo que haga bromas sobre héroes con unas determinadas condiciones, a trabés de la función **Joke**.
- **WritePlugin**: Para que veas que dentro de un plugin puedes tener diferentes funciones, en este directorio tenemos dos relacionadas con el arte de escribir: la primera de ellas, **OOF**, nos permite generar el mensaje de "Fuera de la oficina" 🏢📧 para super héroes y la segunda **StoryGen** nos ayudará a crear historias, también de super héroes 🦸🏻‍♂️🦸🏻‍♀️.

Para que nuestro kernel sepa que estos plugins están disponibles necesitas primero obtener el path del directorio:

In [10]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions");

Y después empezar a cargar los plugins que quieras:

In [11]:
// Load the FunPlugin from the Plugins Directory
var funPluginFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "FunPlugin");

Para poder invocar una función de este tipo puedes hacerlo de la siguiente manera:

In [12]:
var result = await kernel.RunAsync("Cuentame un chiste", funPluginFunctions["Joke"]);

Como ves, tan solo utilizando **kernel.RunAsync** y pasando como parámetros mi petición y cuál es la función, en este caso **Joke**, dentro del plugin, en este caso **FunPlugin**, que quiero utilizar.

Cada una de estas funciones consta de dos archivos:

- **skprompt.txt** es el archivo donde se define el prompt a mandar al modelo en un formato plantilla, de tal forma que puede recibir parámetros y hacerlos más reusables.

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE SUBJECT BELOW

JOKE MUST BE:
- IN SPANISH

BE CREATIVE AND FUNNY. I WANT TO LAUGH.

Incorporate the hero if provided: {{$hero}}
+++++

{{$input}}
+++++
```
- **config.json**, el cual nos permite indicar el número de tokens máximo permitido para esta llamada (**max_tokens**), la temperatura para controlar la aleatoriedad de las respuestas (**temperature**), lo cual significa que más cerca del 1 serán más aleatorias y más cerca del cero más determinadas y centradas en la respuesta más probable, **top_p** se utiliza para controlar la diversidad de las respuestas, donde un valor de 0.0 significa que solo se considerarán las respuestas más probables y 1 donde se considerarán todas las respuestas posibles y **presence_penaltty** y **frecuency_penalty** para ajustar la penalización por las presencia y frecuencia de los tokens en las respuesta generadas. Por otro lado, si el archivo skprompt.txt recibe parámetros se deben definir en este archivo también en el array **parameters** del objeto **input**

```javascript
{
  "schema": 1,
  "description": "Generate a funny joke about heroes",
  "models": [
    {
      "max_tokens": 150,
      "temperature": 0.9,
      "top_p": 0.5,
      "presence_penalty": 0.2,
      "frequency_penalty": 0.3
    }
  ],
  "input": {
    "parameters": [
      {
        "name": "input",
        "description": "Joke subject",
        "defaultValue": ""
      },
      {
        "name": "hero",
        "description": "Give a hint about the hero you want to joke about",
        "defaultValue": ""
      }
    ]
  }
}
```


Para ver el resultado generado de llamar a esta función Joke puedes recuperarlo de la siguiente manera:

In [13]:
Console.WriteLine(result.GetValue<string>());

Claro, aquí tienes un chiste:

¿Por qué Batman se metió al boxeo?

Porque quería ser un "golpe-ador" nocturno.


Como puedes ver, en esta ejecución no le hemos pasado más que el parámetro input, pero no el héroe del cual queríamos generar este chiste. Para poder mandarle más de un parámetro necesitas crear un objeto del tipo **ContextVariables**:

In [17]:
var variables = new ContextVariables{
    ["input"] = "Cuentame un chiste sobre Navidad",
    ["hero"] = "Ironman"
};

Para poder utilizar estos dos valores como parte de la llamara simplemente hay que ponerlo como primer argumento.

In [18]:
var result = await kernel.RunAsync(variables, funPluginFunctions["Joke"]);

Ahora comprueba si el chiste es del héroe especificado como parámetro.

In [19]:
Console.WriteLine(result.GetValue<string>());

¿Por qué Ironman no usa su traje en Navidad?

Porque ya tiene suficiente con el hierro de los turrones.


De la misma forma, podemos utilizar las funciones incluídas en **WriterPlugin**:

In [20]:
// Load the WriterPlugin from the Plugins Directory
var writerPluginFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");

Uno que genera historias:

In [21]:
var result = await kernel.RunAsync("Cuentame una historia sobre las navidades", writerPluginFunctions["StoryGen"]);
Console.WriteLine(result);

Había una vez, en una fría noche de Navidad 🎄, Hulk 💪, nuestro héroe verde y gigante, estaba solo en su casa. A pesar de su fuerza y coraje, sentía un vacío en su corazón 💔.

En medio de la noche 🌙, Hulk escuchó un ruido proveniente del techo. Se levantó y vio a Santa Claus 🎅 atascado en la chimenea. "¡Oh no! ¡Estoy atrapado!", gritó Santa.

Hulk, con su fuerza sobrehumana, liberó a Santa de la chimenea. "¡Gracias, Hulk!", dijo Santa aliviado. Pero había un problema: el trineo 🛷 de Santa estaba roto.

Hulk, decidido a salvar la Navidad, usó sus habilidades para reparar el trineo. Con un poco de esfuerzo y mucha cinta adhesiva, el trineo estaba listo para volar nuevamente.

Santa le agradeció a Hulk y le ofreció un regalo 🎁: "Hulk, has salvado la Navidad. ¿Qué te gustaría como regalo?" Hulk pensó por un momento y luego respondió: "Quiero amigos con quienes compartir estas fiestas".

Santa sonrió y asintió. Esa noche, Hulk no estuvo solo. Los Vengadores se reunieron en su casa para celeb

O incluso para crear mensajes para cuando los héroes 🦸🏻‍♀️ están de vacaciones ✈️🚢🌴:

In [22]:
var result = await kernel.RunAsync("Crea un out of office para los días de Navidades", writerPluginFunctions["OOF"]);
Console.WriteLine(result.GetValue<string>());

¡Hola! 🐾

Soy Catwoman, tu heroína de confianza. Quiero informarte que estaré fuera de la ciudad desde el 24 de diciembre hasta el 1 de enero, disfrutando de unas merecidas vacaciones navideñas. 🎄🎅

Durante mi ausencia, no podré utilizar mis habilidades de agilidad felina, ni mi astucia para resolver los problemas que puedan surgir. Pero no te preocupes, no te dejaré desprotegido. 🦸‍♀️

En mi lugar, Batman estará vigilando la ciudad. Con su fuerza, inteligencia y tecnología avanzada, estoy segura de que podrá manejar cualquier situación que se presente. 🦇💪

Si necesitas ayuda, no dudes en contactar a Batman. Él estará más que dispuesto a ayudarte en mi ausencia. 

Gracias por tu comprensión. Espero que tengas unas felices fiestas y un próspero año nuevo. Nos vemos en enero! 🎉🎊

Adiós y cu


## Native Functions

Si bien las funciones semánticas nos permiten definir y reutilizar prompts, **con las funciones nativas puedes hacer que semantic kernel pueda llamar a funciones escritas en C# o Python**, para tareas que se escapan propiamente de una llamada a través de un prompt.

<div style="text-align:center">
    <img src="images/native-function-explainer.png" width="25%">
</div>

### ¿Por qué necesito funciones nativas en este tipo de aplicaciones?

Los LLM, o Large language models, son excelentes para generar texto, pero hay varias tareas que no pueden realizar por sí solos. Éstas incluyen, entre otras:

- Recuperar datos de fuentes de datos externas
- Saber qué hora es
- Realizar operaciones matemáticas complejas
- Completar tareas en el mundo real
- Memorizar y recordar información

Para estos escenarios, y muchos otros, las funciones nativas son de gran utilidad 👍🏻

Para esto ejemplo, voy a usar una API llamada **SuperHero API**, la cual necesita una API key. Puedes conseguir la misma en su página web: [https://superheroapi.com/](https://superheroapi.com/)

Una vez la tengas, pasasela al prompt que te aparece con la siguiente línea:

In [23]:
var superHeroApiKey = await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Super Hero Api key");

Ahora, para cargar una función nativa, debemos hacerlo de la siguiente manera:

In [24]:
#load "NativeFunctions/GetHeroInfo.cs"

var infoPlugin = kernel.ImportFunctions(new Info(superHeroApiKey.GetClearTextPassword()), "InfoPlugin");

var result = await kernel.RunAsync("catwoman", infoPlugin["GetAlterEgo"]);

En este repo hay otro directorio llamado **NativeFunctions** donde puedes encontrar una clase llamada **GetHeroInfo.cs** en ella hay una función decorada con el atributo **SKFunction** el cual nos permite indicarle a Semantic Kernel que es una función nativa y, a través de la propiedad **Description**, también darle información sobre cuál es el objetivo de esta funcion. En este caso lo que nos permite este método es recuperar el alter ego del super héroe que le pasemos como parámetro, en este ejemplo el de Catwoman. Si echamos un vistazo al resultado podrás comprobar que el mismo es el esperado:

In [25]:
Console.WriteLine(result.GetValue<string>());

Selina Kyle


## Planner

Hasta ahora, todos los plugins que has ido viendo los has ejecutado intencionadamente. Es decir, nadie los ha escogido por tí y tú los puedes ejecutar en base a las necesidades que tengas. Sin embargo, esta es la forma más *estática* de interactuar con Semantic Kernel. Exíste otra opción llamada **Planer** que te va a dejar boquiabierto 😮

Planner es una función que toma la petición de un usuario y devuelve un plan sobre cómo llevar a cabo la solicitud. Para ello, utiliza la IA para combinar los plugins registrados en el núcleo y recombinarlos en una serie de pasos que completen un objetivo.

<div style="text-align:center;">
    <img src="images/the-planner.png" width=35% />
</div>

Para verlo en acción vamos a utilizar los plugins que ya conoces.

Lo primero que necesitas es instanciar un planner:

In [26]:
using Microsoft.SemanticKernel.Planners;

// Create planner
var planner = new SequentialPlanner(kernel);

Hay de diferentes tipos como puedes ver [aquí](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/planners/?tabs=Csharp). En este ejemplo vamos a utilizar el que ejecuta las tareas de forma secuencial.

Como ya tienes todos los plugins cargados en tu instancia del kernel, podemos preguntarle algo como esto:

In [ ]:
using System.Text.Json;

var ask = "Me gustaría que me contaras un chiste sobre Batman, y con el chiste que hicieras un out of office con el chiste.";
var plan = await planner.CreatePlanAsync(ask);

Console.WriteLine("Plan:\n");
Console.WriteLine(JsonSerializer.Serialize(plan, new JsonSerializerOptions { WriteIndented = true }));

Como puedes ver el planificador sabe sobre mis plugins gracias a la descripción que incluí como parte de su implementación.

In [28]:
var result = await kernel.RunAsync(plan);

Console.WriteLine("Plan result:\n");
Console.WriteLine(result.GetValue<string>());

Plan result:

¡Hola! 🦇

Soy Batman, tu héroe de confianza. Actualmente estoy fuera de la oficina, mejorando mis habilidades en un curso de música para perfeccionar mi "Bat-acorde". 🎵🎶

Durante mi ausencia, no te preocupes, la ciudad de Gotham no quedará desprotegida. Mi buen amigo Superman estará vigilando y manteniendo la paz en mi lugar. 🦸‍♂️💪

Si tienes alguna emergencia, no dudes en contactar a Superman. Él tiene la velocidad, la fuerza y la visión de rayos X para manejar cualquier situación que pueda surgir. 🚀🌍

Volveré pronto, más fuerte y más afinado que nunca. Hasta entonces, mantén la calma y confía en Superman. 

Adiós y mantente seguro. 🌃🌙

Batman.


## LABS

El kernel ya tiene cargado los plugins, y la funcion, veamos si puede resolver un ejemplo mas complejo:

In [29]:
using System.Text.Json;

var ask_complex = "Me gustaría que avveriguaras el alter ego de Batman, y que hicieras un out of office para Batman firmando con el alter ego del mismo.";
var plan_complex = await planner.CreatePlanAsync(ask_complex);

Console.WriteLine("Plan:\n");
Console.WriteLine(JsonSerializer.Serialize(plan_complex, new JsonSerializerOptions { WriteIndented = true }));

Plan:

{
  "state": [],
  "steps": [
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "Batman"
        }
      ],
      "outputs": [
        "ALTER_EGO"
      ],
      "next_step_index": 0,
      "name": "GetAlterEgo",
      "plugin_name": "InfoPlugin",
      "description": "Get the alter ego of a superhero",
      "model_settings": [],
      "RequestSettings": null
    },
    {
      "state": [],
      "steps": [],
      "parameters": [
        {
          "Key": "input",
          "Value": "Estoy fuera luchando contra el crimen"
        },
        {
          "Key": "hero",
          "Value": "Batman"
        }
      ],
      "outputs": [
        "OOF_MESSAGE"
      ],
      "next_step_index": 0,
      "name": "OOF",
      "plugin_name": "WriterPlugin",
      "description": "Generate a out of office message for a given hero subject",
      "model_settings": [
        {
          "service_id": null,
          "mod

Ejecutemos el plan y veamos su output:

In [30]:
var result_complex = await kernel.RunAsync(plan_complex);

Console.WriteLine("Plan result:\n");
Console.WriteLine(result_complex.GetValue<string>());

Plan result:

Estimado remitente,

🦇🌃 Actualmente, me encuentro fuera de la oficina, combatiendo el crimen en las oscuras calles de Gotham. Como Batman, mi deber es proteger la ciudad y garantizar la seguridad de sus ciudadanos, lo que a veces requiere de mi atención total y absoluta.

Durante mi ausencia, no te preocupes, no estás desprotegido. 🦸‍♂️💫 Superman estará disponible para asistirte en cualquier emergencia que pueda surgir. Con su velocidad, fuerza y capacidad de volar, estoy seguro de que podrá manejar cualquier situación que se presente.

Agradezco tu comprensión y paciencia durante este tiempo. Recuerda, cada uno de nosotros tiene el poder de hacer una diferencia, por pequeña que sea. 🌟💪

Regresaré tan pronto como Gotham esté segura nuevamente. Hasta entonces, mantén la calma y confía en que el bien siempre prevalecerá.

Adiós y cuídate,

Batman 🦇🌙. Firma: Terry


## Kernel Memory

<img src="images/How kernel memory works.png" width="80%" />

Para este ejemplo voy a utilizar **Open AI** en lugar de Azure Open AI, por lo que necesitas guardar [una API Key](https://platform.openai.com/account/api-keys) de este en la siguiente variable:

Para poder utilizar Kernel Memory necesitas añadir su librería de nuget, además de importa la clase que he generado en el directorio **KernelMemory**.

In [50]:
#r "nuget: Microsoft.KernelMemory.Core, 0.11.231120.6-preview"

#!import "KernelMemory/Memories.cs"

Installed Packages Microsoft.KernelMemory.Core, 0.11.231120.6-preview

Lo primero que voy a hacer es importar a Kernel Memory unos cuantos *recuerdos* (textos) y *documentos*.

In [51]:
var openApiKey = await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Open AI key");

MemoryKernel.Init(openApiKey.GetClearTextPassword());

Error: Command cancelled.

Como puedes ver en el output, este ya se encarga de generar los embeddings de las frases/documentos que le pasamos para que el modelo de GPT-4 pueda generar la respuesta.
Ahora que ya tenemos algo de contenido sobre el que preguntar, vamos a cargar esta clase como un plugin más.

In [46]:
var memoriesPlugin = kernel.ImportFunctions(new MemoryKernel(), "MemoriesPlugin");

Y ahora preguntemos sobre el contenido:

In [47]:
var planner = new SequentialPlanner(kernel);

var plan = await planner.CreatePlanAsync("¿Quién es el héroe favorito de Bruno?");

var result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());


Answer: Mi héroe favorito es Batman.
  - content.txt  - gis01/62eb12bbb53c4a31924fbe94b9a7b4c2 [Thursday, November 30, 2023]
  - content.txt  - bru01/881e2c778cca47a9975fa2caf0aeaf4b [Thursday, November 30, 2023]
  - content.txt  - bru02/2b2d1c65585443b49bbb95236323f5b6 [Thursday, November 30, 2023]
  - content.txt  - gis02/bdca3bc681c44aa3a339d90be33d2db4 [Thursday, November 30, 2023]
{"answer":"Mi h\u00E9roe favorito es Batman.","references":["content.txt","content.txt","content.txt","content.txt"]}


In [48]:
var planner = new SequentialPlanner(kernel);

var plan = await planner.CreatePlanAsync("¿Cuál fue la última película que vio Bruno?");

var result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());

Error: Microsoft.SemanticKernel.Diagnostics.SKException: Not possible to create plan for goal with available functions.
Goal:¿Cuál fue la última película que vio Bruno?
Functions:
FunPlugin.Joke:
  description: Generate a funny joke about heroes
  inputs:
    - input: Joke subject
    - hero: Give the name of the hero you want to joke about

InfoPlugin.GetAlterEgo:
  description: Get the alter ego of a superhero
  inputs:
    - input: 

MemoriesPlugin.AboutMe:
  description: Responde preguntas sobre Bruno
  inputs:
    - ask: 

MemoriesPlugin.AboutTheDocuments:
  description: Sobre el volumen
  inputs:
    - ask: 

WriterPlugin.OOF:
  description: Generate a out of office message for a given hero subject
  inputs:
    - input: The reason for being out of office
    - hero: hero name (default value: Catwoman)

WriterPlugin.StoryGen:
  description: Generate a short story about the hero of your choice.
  inputs:
    - input: story subject
    - hero: hero name (default value: Hulk)
   at

También podemos preguntarle por el PDF que incluí:

In [38]:
var planner = new SequentialPlanner(kernel);

var plan = await planner.CreatePlanAsync("¿Qué incluye este volumen de batman?");

var result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());


Answer: Este volumen de Batman incluye la historia definitiva del Caballero Oscuro en el cómic, el cine y más allá. Detalla toda la historia de Batman y las muchas formas que ha asumido a lo largo de los años, en cómics, televisión, animación, películas, videojuegos y más. Además, incluye comentarios exclusivos de personas creativas que han sido fundamentales para la creación del legado de Batman. También cuenta con documentos extraíbles que ayudan a acentuar la experiencia de lectura.
  - Norma editorial - Septiembre 2020.pdf  - doc001/43b043c48789457abbce9733779e9024 [Thursday, November 30, 2023]
{"answer":"Este volumen de Batman incluye la historia definitiva del Caballero Oscuro en el c\u00F3mic, el cine y m\u00E1s all\u00E1. Detalla toda la historia de Batman y las muchas formas que ha asumido a lo largo de los a\u00F1os, en c\u00F3mics, televisi\u00F3n, animaci\u00F3n, pel\u00EDculas, videojuegos y m\u00E1s. Adem\u00E1s, incluye comentarios exclusivos de personas creativas que h